In [1]:
import secrets
from winternitz_utils import hash_chain 

# Naive 4 Bit Winternitz Signatures

This notebook contains a simple implementation of Winternitz one-time signatures for 4 bit numbers. Because no checksum is used signatures can be forged. Being able to easily forge signatures makes this scheme insecure!

## Generate Key Pair

**Private Key** ($sk$): Random number from 256 bits (or whatever the hash function input length is).

$$sk \overset{{\scriptscriptstyle\$}}{\leftarrow} \mathbb{\{0, 1\}^{256}}$$     

**Public Key** ($pk$): Hash the private key 16 times (as $2^4 = 16$ for 4 bit messages).

$$h(h(h(h(h(h(h(h(h(h(h(h(h(h(h(h(sk)))))))))))))))) = h^{16}(sk) = pk$$

In [2]:
private_key = secrets.token_bytes(32) # 256 bits (SHA3-256 accepts inputs of 256 bits)
public_key = hash_chain(16, private_key)

print(f"{private_key.hex()=}")
print(f"{public_key.hex()=}")

private_key.hex()='a08c4ba46969948613d626c6deb66d722202e7f41baa3d4b3c15da3d3f64a1b8'
public_key.hex()='fc45d633f53380e2baa4aad16c1bcc0fa4626cc7b138fc8b6cffe3b2e146ad8e'


## Sign Message

In order to sign a message we take our private key and hash it $m$ times. In our example our message is 5, therefore we construct a [hash chain](https://en.wikipedia.org/wiki/Hash_chain) of length 5.

When $m=5$ we use the following formula ($h$ is our hash function and $sk$ is the private key):

$$h(h(h(h(h(sk))))) = h^5(sk) = h^m(sk) = sig^5$$ 

In [3]:
message = 5 #0b0101
signature = hash_chain(message, private_key)

print(f"{signature.hex()=}")

signature.hex()='1a8244737de949d5044a35ccc56b8f7c29b638fe740c3b876b17eac6742c75ab'


## Verify Message

To verify a signed message the verifier must know the public key and the message. The verifier then hashes the signature $16 - m$ times and compares it to the public key. If the public key and the hashed signature match then the signature is valid. The following formula shows how the verifier can check the signature ($h$ is our hash function, $sk$ is the private key and $pk$ is the public key):

$$h^{16 - m}(sig^5) = h^{16 - m}(h^m(sk)) = h^{16 - m + m}(sk)= h^{16}(sk) = pk$$ 

We use the following formula to check if the signature is valid:

$$h^{16 - m}(sig^5) \equiv pk$$

In [4]:
# If true then the signature is valid
hash_chain(16 - message, signature) == public_key

True

## Forged Signature

Because a one-way function is easy to compute in the forward direction we can easily forge proofs for messages where the forged messaged is larger than $m$. For example if we sign a message of 5, an adversary can forge a proof for a message of 6. This is can be done using the following formula:

$$h(sig^5) = h(h^5(sk)) = h^6(sk) = sig^6$$

The [next notebook](https://github.com/Blake-Haydon/Winternitz-Signatures/blob/main/4_bit_winternitz_secure.ipynb) contains a fix for this problem.

In [5]:
forged_message = 6 #0b0110
assert forged_message > message # The forged message must be greater than the original message

# Successfully forge signature of new message
forged_signature = hash_chain(forged_message - message, signature)
hash_chain(16 - forged_message, forged_signature) == public_key

True